Based on the assembletics analysis in Pst_104E_v12_assembletics_analysis on fisher. This script aims to generate some figures for the paper showing two dot plots of two contigs
Do plots for the following contigs
pcontig_002, pcontig_019, pcontig_028

In [127]:
%matplotlib inline
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SearchIO
import shutil

In [128]:
NUCMER_FOLDER_IN = '/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/'
NUCMER_FOLDER_OUT = '/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/'

In [129]:
#define file names
pcontig_002_fn = 'Pst_104E_v12_pcontig_002.fa'
pcontig_002_h_fn = 'Pst_104E_v12_pcontig_002_h_ctgs.fa'
pcontig_019_fn = 'Pst_104E_v12_pcontig_019.fa'
pcontig_019_h_fn = 'Pst_104E_v12_pcontig_019_h_ctgs.fa'
pcontig_028_fn = 'Pst_104E_v12_pcontig_028.fa'
pcontig_028_h_fn = 'Pst_104E_v12_pcontig_028_h_ctgs.fa'

In [130]:
#now read in the h files and order them accordingly
h_002_fh = open(os.path.join(NUCMER_FOLDER_IN, pcontig_002_h_fn), 'r')
order = ['hcontig_002_011', 'hcontig_002_023', 'hcontig_002_029', 'hcontig_002_028', 'hcontig_002_004'\
         ,'hcontig_002_021','hcontig_002_019']
out_list = [0]*len(order)
index_dict = {}
for seq in SeqIO.parse(h_002_fh, 'fasta'):
    out_list[order.index(seq.id)] = seq
h_002_fh.close()

In [131]:
#write out files and move files to output folder
h_002_out_fh = open(os.path.join(NUCMER_FOLDER_OUT, pcontig_002_h_fn), 'w')
SeqIO.write(out_list, h_002_out_fh, 'fasta')
h_002_out_fh.close()
shutil.copyfile(os.path.join(NUCMER_FOLDER_IN, pcontig_002_fn), os.path.join(NUCMER_FOLDER_OUT, pcontig_002_fn))

'/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_v12_pcontig_002.fa'

In [148]:
#do the nucmer plots
os.chdir(NUCMER_FOLDER_OUT)
ref = pcontig_002_fn
query = pcontig_002_h_fn
!nucmer --maxgap=500 --mincluster=100 --prefix={ref[:-3]} {ref} {query}
!show-coords -r {ref[:-3]}.delta > {ref[:-3]}.coords
!delta-filter -q -r {ref[:-3]}.delta > {ref[:-3]}.filter
!mummerplot --postscript {ref[:-3]}.filter -R {ref} -Q {query} --prefix={ref[:-3]}

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "Pst_104E_v12_pcontig_002.ntref" of length 2342973
# construct suffix tree for sequence of length 2342973
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 23429 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /Users/ben/anaconda3/opt/mummer-3.23/mummer Pst_104E_v12_pcontig_002.ntref 1.31
# reading input file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_v12_pcontig_002_h_ctgs.fa" of length 2461430
# matching query-file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_

In [158]:
#now fix the gnuplotscript in for pcontig_002 specifically
ref = pcontig_002_fn
query = pcontig_002_h_fn
new_gp = ref.replace('.fa', '_v1.gp')
new_gp_fh = open(new_gp, 'w')
with open(ref.replace('.fa', '.gp')) as f:
          for line in f:
              if '"Courier" 8' in line:
                    new_gp_fh.write(line.replace('"Courier" 8', '"Courier" 22'))
              elif 'line 1' in line:
                  new_gp_fh.write("set style line 1  lt 3 lw 6 pt 0.5 ps 0.5 lc rgb '#000004'\n")
              elif '023' in line or '004' in line or '021' in line or '019' in line:
                  continue
              elif   '931489' in line:
                  new_gp_fh.write(line.replace('931489', '900000').replace('hcontig_', ''))
              elif   '968280' in line:
                  new_gp_fh.write(line.replace('968280', '1000000').replace('hcontig_', ''))     
              elif 'hcontig' in line:
                  new_gp_fh.write(line.replace('hcontig_', ''))
              elif 'xlabel' in line:
                  new_gp_fh.write('set xlabel "primary contig %s\n' %re.search(r'g_[0-9]*', new_gp).group(0)[2:])
              elif '"QRY"' in line:
                  new_gp_fh.write('set ylabel "haplotigs"\n')
              elif 'line 2' in line:
                  new_gp_fh.write("set style line 2  lt 2 lw 6 pt 0.5 ps 0.5 lc rgb '#e35933'\n")
              elif 'set border' in line:
                  new_gp_fh.write(line.replace('10', '15'))
              elif 'set tics scale' in line: 
                  new_gp_fh.write(line.replace('0', '2'))
              else:
                  new_gp_fh.write(line)
          
new_gp_fh.close()
!gnuplot {new_gp}

In [157]:
!gnuplot {new_gp}

In [150]:
#now read in the h files and order them accordingly for p_contig_019
h_019_fh = open(os.path.join(NUCMER_FOLDER_IN, pcontig_019_h_fn), 'r')
order = ['hcontig_019_085','hcontig_019_008','hcontig_019_015','hcontig_019_013','hcontig_019_009','hcontig_019_012'\
        ,'hcontig_019_001','hcontig_019_011','hcontig_019_020','hcontig_019_019']
out_list = [0]*len(order)
index_dict = {}
for seq in SeqIO.parse(h_019_fh, 'fasta'):
    #now reverse complement contig hcontig_019_085
    if seq.id == 'hcontig_019_085':
        seq = seq.reverse_complement()
        seq.id = 'hcontig_019_085'
    print(seq.id)
    out_list[order.index(seq.id)] = seq
h_019_fh.close()

hcontig_019_085
hcontig_019_001
hcontig_019_008
hcontig_019_009
hcontig_019_011
hcontig_019_012
hcontig_019_013
hcontig_019_015
hcontig_019_019
hcontig_019_020


In [151]:
#write out files and move files to output folder
h_019_out_fh = open(os.path.join(NUCMER_FOLDER_OUT, pcontig_019_h_fn), 'w')
SeqIO.write(out_list, h_019_out_fh, 'fasta')
h_019_out_fh.close()
shutil.copyfile(os.path.join(NUCMER_FOLDER_IN, pcontig_019_fn), os.path.join(NUCMER_FOLDER_OUT, pcontig_019_fn))

'/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_v12_pcontig_019.fa'

In [152]:
#do the nucmer plots
os.chdir(NUCMER_FOLDER_OUT)
ref = pcontig_019_fn
query = pcontig_019_h_fn
!nucmer --maxgap=500 --mincluster=100 --prefix={ref[:-3]} {ref} {query}
!show-coords -r {ref[:-3]}.delta > {ref[:-3]}.coords
!delta-filter -q -r {ref[:-3]}.delta > {ref[:-3]}.filter
!mummerplot --postscript {ref[:-3]}.filter -R {ref} -Q {query} --prefix={ref[:-3]}

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "Pst_104E_v12_pcontig_019.ntref" of length 1435106
# construct suffix tree for sequence of length 1435106
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 14351 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /Users/ben/anaconda3/opt/mummer-3.23/mummer Pst_104E_v12_pcontig_019.ntref 0.72
# reading input file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_v12_pcontig_019_h_ctgs.fa" of length 1797113
# matching query-file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_

In [159]:
#now fix the gnuplotscript for 019
ref = pcontig_019_fn
query = pcontig_019_h_fn
new_gp = ref.replace('.fa', '_v1.gp')
new_gp_fh = open(new_gp, 'w')
with open(ref.replace('.fa', '.gp')) as f:
          for line in f:
              if '"Courier" 8' in line:
                    new_gp_fh.write(line.replace('"Courier" 8', '"Courier"  22'))
              elif 'line 1' in line:
                  new_gp_fh.write("set style line 1  lt 3 lw 6 pt 0.5 ps 0.5 lc rgb '#000004'\n")
              elif '011' in line :
                  continue    
              elif 'hcontig' in line:
                  new_gp_fh.write(line.replace('hcontig_', ''))
              elif 'xlabel' in line:
                  new_gp_fh.write('set xlabel "primary contig %s\n' %re.search(r'g_[0-9]*', new_gp).group(0)[2:])
              elif '"QRY"' in line:
                  new_gp_fh.write('set ylabel "haplotigs"\n')
              elif 'line 2' in line:
                  new_gp_fh.write("set style line 2  lt 2 lw 6 pt 0.5 ps 0.5 lc rgb '#e35933'\n")
              elif 'set border' in line:
                  new_gp_fh.write(line.replace('10', '15'))
              elif 'set tics scale' in line: 
                  new_gp_fh.write(line.replace('0', '2'))
              else:
                  new_gp_fh.write(line)
          
new_gp_fh.close()
!gnuplot {new_gp}

In [168]:
!gnuplot Pst_104E_v12_pcontig_019_manual_v1.gp

In [113]:
#now read in the h files and order them accordingly for p_contig_019
h_028_fh = open(os.path.join(NUCMER_FOLDER_IN, pcontig_028_h_fn), 'r')
order = ['hcontig_028_011', 'hcontig_028_018','hcontig_028_015','hcontig_028_005','hcontig_028_003','hcontig_028_007']
out_list = [0]*len(order)
index_dict = {}
for seq in SeqIO.parse(h_028_fh, 'fasta'):
    print(seq.id)
    out_list[order.index(seq.id)] = seq
h_028_fh.close()

hcontig_028_003
hcontig_028_005
hcontig_028_007
hcontig_028_011
hcontig_028_015
hcontig_028_018


In [114]:
#write out files and move files to output folder
h_028_out_fh = open(os.path.join(NUCMER_FOLDER_OUT, pcontig_028_h_fn), 'w')
SeqIO.write(out_list, h_028_out_fh, 'fasta')
h_028_out_fh.close()
shutil.copyfile(os.path.join(NUCMER_FOLDER_IN, pcontig_028_fn), os.path.join(NUCMER_FOLDER_OUT, pcontig_028_fn))

'/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_v12_pcontig_028.fa'

In [115]:
#do the nucmer plots
os.chdir(NUCMER_FOLDER_OUT)
ref = pcontig_028_fn
query = pcontig_028_h_fn
!nucmer --maxgap=500 --mincluster=100 --prefix={ref[:-3]} {ref} {query}
!show-coords -r {ref[:-3]}.delta > {ref[:-3]}.coords
!delta-filter -q -r {ref[:-3]}.delta > {ref[:-3]}.filter
!mummerplot --postscript {ref[:-3]}.filter -R {ref} -Q {query} --prefix={ref[:-3]}

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "Pst_104E_v12_pcontig_028.ntref" of length 1135312
# construct suffix tree for sequence of length 1135312
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 11353 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /Users/ben/anaconda3/opt/mummer-3.23/mummer Pst_104E_v12_pcontig_028.ntref 0.55
# reading input file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_v12_pcontig_028_h_ctgs.fa" of length 1217344
# matching query-file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/nucmer_analysis/nucmer_plots/Pst_104E_

In [169]:
#now fix the gnuplotscript in general 
ref = pcontig_028_fn
query = pcontig_028_h_fn
new_gp = ref.replace('.fa', '_v1.gp')
new_gp_fh = open(new_gp, 'w')
with open(ref.replace('.fa', '.gp')) as f:
          for line in f:
              if '"Courier" 8' in line:
                    new_gp_fh.write(line.replace('"Courier" 8', '"Courier"  22'))
              elif 'line 1' in line:
                  new_gp_fh.write("set style line 1  lt 3 lw 6 pt 0.5 ps 0.5 lc rgb '#000004'\n")
              elif '005' in line or '018' in line:
                  continue    
                    
              elif 'hcontig' in line:
                  new_gp_fh.write(line.replace('hcontig_', ''))
              elif 'xlabel' in line:
                  new_gp_fh.write('set xlabel "primary contig %s\n' %re.search(r'g_[0-9]*', new_gp).group(0)[2:])
              elif '"QRY"' in line:
                  new_gp_fh.write('set ylabel "haplotigs"\n')
              elif 'line 2' in line:
                  new_gp_fh.write("set style line 2  lt 2 lw 6 pt 0.5 ps 0.5 lc rgb '#e35933'\n")
              elif 'set border' in line:
                  new_gp_fh.write(line.replace('10', '15'))
              elif 'set tics scale' in line: 
                  new_gp_fh.write(line.replace('0', '2'))
              else:
                  new_gp_fh.write(line)
          
new_gp_fh.close()
!gnuplot {new_gp}

In [170]:
!gnuplot {new_gp}

In [68]:
!gnuplot {new_gp}

Based on the allele shuffling analysis of post allele analysis on fisher and the workbook1 in the post allele analysis on the labtop this part of the script does an alignemnt in order to generate a Ribbon circa plot illustarting large scale transpositions.
For this I will align pcontig_14 with hcontigs_14 and hcontigs_27 + pcontig_17 with hcontig_27 and hcontigs_14

In [171]:
CIRCA_ALN_OUT = '/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln'

In [172]:
if not os.path.exists(CIRCA_ALN_OUT):
    os.mkdir(CIRCA_ALN_OUT)

In [173]:
#define file names of fa files containg the sequences for aligments
pcontig_014_fn = 'Pst_104E_v12_pcontig_014.fa'
pcontig_014_h_fn = 'Pst_104E_v12_pcontig_014_h_ctgs.fa'
pcontig_027_fn = 'Pst_104E_v12_pcontig_027.fa'
pcontig_027_h_fn = 'Pst_104E_v12_pcontig_027_h_ctgs.fa'

In [174]:
#now read in the h files and order them accordingly
h_014_fh = open(os.path.join(NUCMER_FOLDER_IN, pcontig_014_h_fn), 'r')
order = ['hcontig_014_008', 'hcontig_014_022', 'hcontig_014_117', 'hcontig_014_004','hcontig_014_013'\
         ,'hcontig_014_027','hcontig_014_024','hcontig_014_192','hcontig_014_017','hcontig_014_012','hcontig_014_023']
out_list = [0]*len(order)
index_dict = {}
for seq in SeqIO.parse(h_014_fh, 'fasta'):
    #now reverse complement contig hcontig_019_085
    if seq.id == 'hcontig_014_192':
        seq = seq.reverse_complement()
        seq.id = 'hcontig_014_192'
    out_list[order.index(seq.id)] = seq
h_014_fh.close()
#write out files and move files to output folder
h_014_out_fh = open(os.path.join(CIRCA_ALN_OUT , pcontig_014_h_fn), 'w')
SeqIO.write(out_list, h_014_out_fh, 'fasta')
h_014_out_fh.close()
shutil.copyfile(os.path.join(NUCMER_FOLDER_IN, pcontig_014_fn), os.path.join(CIRCA_ALN_OUT , pcontig_014_fn))

'/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln/Pst_104E_v12_pcontig_014.fa'

In [175]:
#do the nucmer plots
os.chdir(CIRCA_ALN_OUT)
ref = pcontig_014_fn
query = pcontig_014_h_fn
!nucmer --maxgap=500 --mincluster=100 --prefix={ref[:-3]} {ref} {query}
!show-coords -T -r {ref[:-3]}.delta > {ref[:-3]}.coords
!delta-filter -q -r {ref[:-3]}.delta > {ref[:-3]}.filter
#added new line to get the coords from the filtered file
!show-coords -T -r {ref[:-3]}.filter > {ref[:-3]}.filter.coords
!mummerplot --postscript {ref[:-3]}.filter -R {ref} -Q {query} --prefix={ref[:-3]}

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "Pst_104E_v12_pcontig_014.ntref" of length 1609766
# construct suffix tree for sequence of length 1609766
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 16097 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /Users/ben/anaconda3/opt/mummer-3.23/mummer Pst_104E_v12_pcontig_014.ntref 0.62
# reading input file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln/Pst_104E_v12_pcontig_014_h_ctgs.fa" of length 1453578
# matching query-file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln/Pst_104E_v12_pcontig_014_h_ctgs.fa"
# against s

In [179]:
#now fix the gnuplotscript for 019
ref = pcontig_014_fn
query = pcontig_014_h_fn
new_gp = ref.replace('.fa', '_v1.gp')
new_gp_fh = open(new_gp, 'w')
with open(ref.replace('.fa', '.gp')) as f:
          for line in f:
              if '"Courier" 8' in line:
                    new_gp_fh.write(line.replace('"Courier" 8', '"Courier"  22'))
              elif 'line 1' in line:
                  new_gp_fh.write("set style line 1  lt 3 lw 6 pt 0.5 ps 0.5 lc rgb '#000004'\n")
              elif '023' in line :
                  continue    
              elif 'hcontig' in line:
                  new_gp_fh.write(line.replace('hcontig_', ''))
              elif 'xlabel' in line:
                  new_gp_fh.write('set xlabel "primary contig %s\n' %re.search(r'g_[0-9]*', new_gp).group(0)[2:])
              elif '"QRY"' in line:
                  new_gp_fh.write('set ylabel "haplotigs"\n')
              elif 'line 2' in line:
                  new_gp_fh.write("set style line 2  lt 2 lw 6 pt 0.5 ps 0.5 lc rgb '#e35933'\n")
              elif 'set border' in line:
                  new_gp_fh.write(line.replace('10', '15'))
              elif 'set tics scale' in line: 
                  new_gp_fh.write(line.replace('0', '2'))
              
              else:
                  new_gp_fh.write(line)
          
new_gp_fh.close()
!gnuplot {new_gp}

In [184]:
!gnuplot Pst_104E_v12_pcontig_014_manual_v1.gp

Prepare for Circa plot by cating together files and making bedpe files

In [83]:
#now order the output file of the haplotigs in revers and remove contigs that have no filtered alignments
#open the initial h_contig input file
h_014_fh = open(os.path.join(NUCMER_FOLDER_IN, pcontig_014_h_fn), 'r')
pcontig_014_h_circa_fn = pcontig_014_h_fn.replace('.fa', '_circa.fa')
#reverse the list and remove the contig without alignment
order_c = order[::-1]
order_c.remove('hcontig_014_023')
out_list_c = len(order_c)*[0]
for seq in SeqIO.parse(h_014_fh, 'fasta'):
    #now reverse complement contig hcontig_014_192
    if seq.id == 'hcontig_014_192':
        seq = seq.reverse_complement()
        seq.id = 'hcontig_014_192'
    #skip over the contig without filtered alignment
    if seq.id == 'hcontig_014_023':
        continue
    out_list_c[order_c.index(seq.id)] = seq
h_014_fh.close()
#open the outfile
h_014_out_fh = open(os.path.join(CIRCA_ALN_OUT , pcontig_014_h_circa_fn), 'w')
SeqIO.write(out_list_c, h_014_out_fh, 'fasta')
h_014_out_fh.close()


In [84]:
min_ribon = 5000
coords_fn = 'Pst_104E_v12_pcontig_014.filter.coords'
mummer_df_tmp = pd.read_csv(coords_fn, sep='\t', header=None,skiprows=4 )
header_mum = ["S1","E1","S2", "E2","LEN_1", "LEN_2", "% IDY", "ID_primary", "ID_haplotig"]
rename = dict(zip(mummer_df_tmp.columns, header_mum))
mummer_df_tmp.rename(columns=rename,inplace=True)

In [85]:
#make bedpe file after filtering down by mimum alingment size
bedpe_fn = coords_fn.replace('.coords', '_%i.bedpe' % min_ribon)
mummer_df_tmp[(mummer_df_tmp.LEN_1 > min_ribon) & (mummer_df_tmp.LEN_2 > min_ribon) ]\
.loc[:, ["ID_primary", 'S1', 'E1', "ID_haplotig", 'S2','E2']]\
.to_csv(bedpe_fn, index=None, sep='\t')

In [86]:
#now fix up the files for the circa plot
os.chdir(CIRCA_ALN_OUT)
#filenames
all_contigs_fn = pcontig_014_fn.replace('.fa', '_all_contigs.fa')
all_contigs_circa_gfn = all_contigs_fn.replace('.fa', '_circa.genome_file')

!cat {pcontig_014_fn} {pcontig_014_h_circa_fn} > {all_contigs_fn}
#remove contigs stuff in names
!sed 's/pcontig_//g' {all_contigs_fn} | sed 's/hcontig_//g' > {all_contigs_fn.replace('.fa', '_circa.fa')}
!samtools faidx {all_contigs_fn.replace('.fa', '_circa.fa')}
!echo 'Contig\tLength' > {all_contigs_circa_gfn}
!cut -f 1,2 {all_contigs_fn.replace('.fa', '_circa.fa.fai')} >> {all_contigs_circa_gfn} 

#also fix up the bedpe file structure

!sed 's/pcontig_//g' {bedpe_fn} | sed 's/hcontig_//g' > {bedpe_fn.replace('.bedpe', '_circa.bedpe')}

In [ ]:
#now do the plotting for hcontig_27 on 14


In [186]:
#now read in the h files and order them accordingly
h_027_fh = open(os.path.join(NUCMER_FOLDER_IN, pcontig_027_h_fn), 'r')
order = ['hcontig_027_102', 'hcontig_027_006', 'hcontig_027_001', 'hcontig_027_018','hcontig_027_011']
out_list = [0]*len(order)
index_dict = {}
for seq in SeqIO.parse(h_027_fh, 'fasta'):
    #now reverse complement contig hcontig_019_085
    out_list[order.index(seq.id)] = seq
h_027_fh.close()
#write out files and move files to output folder
h_027_out_fh = open(os.path.join(CIRCA_ALN_OUT , pcontig_027_h_fn), 'w')
SeqIO.write(out_list, h_027_out_fh, 'fasta')
h_027_out_fh.close()
shutil.copyfile(os.path.join(NUCMER_FOLDER_IN, pcontig_027_fn), os.path.join(CIRCA_ALN_OUT , pcontig_027_fn))

'/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln/Pst_104E_v12_pcontig_027.fa'

In [187]:
#do the nucmer plots
os.chdir(CIRCA_ALN_OUT)
ref = pcontig_027_fn
query = pcontig_027_h_fn
!nucmer --maxgap=500 --mincluster=100 --prefix={ref[:-3]} {ref} {query}
!show-coords -T -r {ref[:-3]}.delta > {ref[:-3]}.coords
!delta-filter -q -r {ref[:-3]}.delta > {ref[:-3]}.filter
#added new line to get the coords from the filtered file
!show-coords -T -r {ref[:-3]}.filter > {ref[:-3]}.filter.coords
!mummerplot --postscript {ref[:-3]}.filter -R {ref} -Q {query} --prefix={ref[:-3]}

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "Pst_104E_v12_pcontig_027.ntref" of length 1172541
# construct suffix tree for sequence of length 1172541
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 11725 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /Users/ben/anaconda3/opt/mummer-3.23/mummer Pst_104E_v12_pcontig_027.ntref 0.50
# reading input file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln/Pst_104E_v12_pcontig_027_h_ctgs.fa" of length 1109675
# matching query-file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln/Pst_104E_v12_pcontig_027_h_ctgs.fa"
# against s

In [188]:
#now fix the gnuplotscript for 027
ref = pcontig_027_fn
query = pcontig_027_h_fn
new_gp = ref.replace('.fa', '_v1.gp')
new_gp_fh = open(new_gp, 'w')
with open(ref.replace('.fa', '.gp')) as f:
          for line in f:
              if '"Courier" 8' in line:
                    new_gp_fh.write(line.replace('"Courier" 8', '"Courier"  22'))
              elif 'line 1' in line:
                  new_gp_fh.write("set style line 1  lt 3 lw 6 pt 0.5 ps 0.5 lc rgb '#000004'\n")
              elif '001' in line :
                  continue    
              elif 'hcontig' in line:
                  new_gp_fh.write(line.replace('hcontig_', ''))
              elif 'xlabel' in line:
                  new_gp_fh.write('set xlabel "primary contig %s\n' %re.search(r'g_[0-9]*', new_gp).group(0)[2:])
              elif '"QRY"' in line:
                  new_gp_fh.write('set ylabel "haplotigs"\n')
              elif 'line 2' in line:
                  new_gp_fh.write("set style line 2  lt 2 lw 6 pt 0.5 ps 0.5 lc rgb '#e35933'\n")
              elif 'set border' in line:
                  new_gp_fh.write(line.replace('10', '15'))
              elif 'set tics scale' in line: 
                  new_gp_fh.write(line.replace('0', '2'))
              else:
                  new_gp_fh.write(line)
          
new_gp_fh.close()
!gnuplot {new_gp}

In [ ]:
!gnuplot {new_gp}

In [189]:
#now plot haplotigs of the 027 onto 014
#do the nucmer plots
os.chdir(CIRCA_ALN_OUT)
ref = pcontig_014_fn
query = pcontig_027_h_fn
out_name = ref[:-3] + '_hcontig27'
!nucmer --maxgap=500 --mincluster=100 --prefix={out_name}  {ref} {query}
!show-coords -T -r {out_name}.delta > {out_name}.coords
!delta-filter -q -r {out_name}.delta > {out_name}.filter
#added new line to get the coords from the filtered file
!show-coords -T -r {out_name}.filter > {out_name}.filter.coords
!mummerplot --postscript {out_name}.filter -R {ref} -Q {query} --prefix={out_name}

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "Pst_104E_v12_pcontig_014_hcontig27.ntref" of length 1609766
# construct suffix tree for sequence of length 1609766
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 16097 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /Users/ben/anaconda3/opt/mummer-3.23/mummer Pst_104E_v12_pcontig_014_hcontig27.ntref 0.78
# reading input file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln/Pst_104E_v12_pcontig_027_h_ctgs.fa" of length 1109675
# matching query-file "/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_104E_v12/circa/aln/Pst_104E_v12_pcontig_027_h_

In [190]:
#now fix the gnuplotscript for 027
new_gp = out_name+'_v1.gp'
new_gp_fh = open(new_gp, 'w')
with open(out_name+'.gp') as f:
          for line in f:
              if '"Courier" 8' in line:
                    new_gp_fh.write(line.replace('"Courier" 8', '"Courier"  22'))
              elif 'line 1' in line:
                  new_gp_fh.write("set style line 1  lt 3 lw 6 pt 0.5 ps 0.5 lc rgb '#000004'\n")
              elif '001' in line :
                  continue    
              elif 'hcontig_' in line:
                  new_gp_fh.write(line.replace('hcontig_', ''))
              elif 'xlabel' in line:
                  new_gp_fh.write('set xlabel "primary contig %s\n' %re.search(r'g_[0-9]*', new_gp).group(0)[2:])
              elif '"QRY"' in line:
                  new_gp_fh.write('set ylabel "haplotigs"\n')
              elif 'line 2' in line:
                  new_gp_fh.write("set style line 2  lt 2 lw 6 pt 0.5 ps 0.5 lc rgb '#e35933'\n")
              elif 'set border' in line:
                  new_gp_fh.write(line.replace('10', '15'))
              elif 'set tics scale' in line: 
                  new_gp_fh.write(line.replace('0', '2'))
              else:
                  new_gp_fh.write(line)
          
new_gp_fh.close()
!gnuplot {new_gp}

In [191]:
!gnuplot Pst_104E_v12_pcontig_014_hcontig27_manual_v1.gp